In [1]:
import pandas as pd
import gc
import os
from tqdm import tqdm


In [4]:
# ▶ Base 경로
base_path = "마이박스 다운로드\\최종EDA_unique제거 전체\\train\\"

# ▶ 병합 대상 파일 목록
file_names = [
    "train_1_회원정보_모델링용.csv",
    "train_2_신용정보_모델링용.csv",
    "train_3_매출정보_unique2_학습용.csv",
    "train_4.청구입금정보_변동없음.csv",
    # "train_5.잔액정보_EDA.csv",
    # "train_6.채널정보_unique.csv",
    # "train_7_마케팅정보_unique.csv",
    # "train_8_성과정보_변동없음_결측치처리.csv"
]
file_paths = [base_path + fname for fname in file_names]

# ▶ 데이터 타입 최적화 함수
def optimize_types(df):
    for col in df.select_dtypes(include='number').columns:
        if 'float' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif 'int' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include='object').columns:
        if df[col].nunique() / len(df) < 0.5:
            df[col] = df[col].astype('category')
    return df

# ▶ 저장 경로
step_path = "merged_step.csv"
final_path = "merged_final_concat.csv"

# ▶ 재시작 여부 판단
if os.path.exists(step_path):
    print("✅ 이전 병합 파일을 불러와 이어서 진행합니다.")
    merged = pd.read_csv(step_path)
    start_idx = len(merged.columns) - 1  # ID 포함
else:
    print("🆕 새로운 병합을 시작합니다.")
    df = pd.read_csv(file_paths[0])
    df = optimize_types(df)
    merged = df.copy()
    merged.to_csv(step_path, index=False)
    print(f"✅ 첫 파일 저장 완료: {step_path}")
    del df
    gc.collect()
    start_idx = 1  # 첫 파일 이후부터 시작

# ▶ tqdm으로 병합 진행
for i in tqdm(range(start_idx, len(file_paths)), desc="📁 파일 병합 중", unit="file"):
    file = file_paths[i]
    print(f"\n📄 병합 중: {file_names[i]}")
    df = pd.read_csv(file)
    df = optimize_types(df)

    merged = pd.concat([merged, df.drop(columns=['ID'], errors='ignore')], axis=1)
    merged.to_csv(step_path, index=False)
    print(f"✅ 중간 저장 완료: {step_path} (열 {merged.shape[1]})")

    del df
    gc.collect()

# ▶ 중복 컬럼 제거 (ID 등)
merged = merged.loc[:, ~merged.columns.duplicated()]

# ▶ 최종 저장
merged.to_csv(final_path, index=False)
print(f"\n🎉 최종 병합 완료: {final_path} (행 {merged.shape[0]}, 열 {merged.shape[1]})")


🆕 새로운 병합을 시작합니다.
✅ 첫 파일 저장 완료: merged_step.csv


📁 파일 병합 중:   0%|          | 0/3 [00:00<?, ?file/s]


📄 병합 중: train_2_신용정보_모델링용.csv
✅ 중간 저장 완료: merged_step.csv (열 104)

📄 병합 중: train_3_매출정보_unique2_학습용.csv
✅ 중간 저장 완료: merged_step.csv (열 444)

📄 병합 중: train_4.청구입금정보_변동없음.csv
✅ 중간 저장 완료: merged_step.csv (열 487)

🎉 최종 병합 완료: merged_final_concat.csv (행 2400000, 열 484)


In [4]:
print(df.shape)
# → (2400000, 871) 정도가 나와야 정상

(2400000, 341)


# 혹시 모르니 하나씩 열어서 확인...

In [3]:
# ▶ Base 경로
base_path = "마이박스 다운로드\\최종EDA_unique제거 전체\\train\\"

# ▶ 병합 대상 파일 목록
file_names = [
    "train_1_회원정보_모델링용.csv",
    "train_2_신용정보_모델링용.csv",
    "train_3_매출정보_unique2_학습용.csv",
    "train_4.청구입금정보_변동없음.csv",
    "train_5.잔액정보_EDA.csv",
    "train_6.채널정보_unique.csv",
    "train_7_마케팅정보_unique.csv",
    "train_8_성과정보_변동없음_결측치처리.csv"
]

# ▶ 파일별 행/열 개수 확인
for fname in file_names:
    path = base_path + fname
    try:
        df = pd.read_csv(path)
        print(f"✅ {fname}: {df.shape[0]:,} rows × {df.shape[1]:,} columns")
    except Exception as e:
        print(f"❌ {fname}: 로딩 실패\n  └ 오류 메시지: {e}")


데이터셋 크기: (2400000, 65)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,남녀구분코드,int64,0,2,2,1,1
3,회원여부_이용가능,int64,0,2,1,1,1
4,회원여부_이용가능_CA,int64,0,2,1,1,1
...,...,...,...,...,...,...,...
60,카드신청건수,int64,0,2,0,0,0
61,Life_Stage,object,0,7,자녀성장(2),자녀성장(1),자녀출산기
62,최종카드발급경과월,int64,0,68,22,18,20
63,연령_num,int64,0,6,45,35,35


✅ train_1_회원정보_모델링용.csv: 2,400,000 rows × 65 columns
데이터셋 크기: (2400000, 40)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,최초한도금액,int64,0,8802,0,0,0
3,카드이용한도금액,int64,0,81852,19354,9996,88193
4,CA한도금액,int64,0,63634,7270,5718,35207
5,일시상환론한도금액,int64,0,7307,0,41996,0
6,월상환론한도금액,int64,0,65045,0,90611,0
7,CA이자율_할인전,float64,0,1882014,22.995207,14.793821,22.014276
8,CL이자율_할인전,float64,0,2126771,18.254978,14.834873,17.875321
9,RV일시불이자율_할인전,float64,0,1897473,17.264967,10.622446,17.155829


✅ train_2_신용정보_모델링용.csv: 2,400,000 rows × 40 columns
데이터셋 크기: (2400000, 341)


...

✅ train_6.채널정보_unique.csv: 2,400,000 rows × 66 columns
데이터셋 크기: (2400000, 40)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,컨택건수_카드론_TM_B0M,int64,0,4,0,3,0
3,컨택건수_리볼빙_TM_B0M,int64,0,3,0,0,0
4,컨택건수_이용유도_TM_B0M,int64,0,3,0,0,0
5,컨택건수_보험_TM_B0M,int64,0,2,0,0,0
6,컨택건수_카드론_LMS_B0M,int64,0,2,0,0,0
7,컨택건수_CA_LMS_B0M,int64,0,3,0,0,0
8,컨택건수_리볼빙_LMS_B0M,int64,0,2,0,0,0
9,컨택건수_이용유도_LMS_B0M,int64,0,4,2,0,0


✅ train_7_마케팅정보_unique.csv: 2,400,000 rows × 40 columns
데이터셋 크기: (2400000, 49)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,증감율_이용건수_신용_전월,float64,0,1497553,0.454548,-0.511808,-0.062716
3,증감율_이용건수_신판_전월,float64,0,1496398,0.454548,-0.511808,-0.062716
4,증감율_이용건수_일시불_전월,float64,0,1494269,0.364727,-0.511808,0.067843
5,증감율_이용건수_할부_전월,float64,0,55263,1.999996,0.0,-1.999996
6,증감율_이용건수_CA_전월,float64,0,12236,0.0,0.0,0.0
7,증감율_이용건수_체크_전월,float64,0,345407,0.0,0.0,0.0
8,증감율_이용건수_카드론_전월,float64,0,2,0.0,0.0,0.0
9,증감율_이용금액_신용_전월,float64,0,1636476,0.409766,-1.010817,-0.902166


✅ train_8_성과정보_변동없음_결측치처리.csv: 2,400,000 rows × 49 columns


- 5번 파일 다시 받기